In [80]:
import autogluon as ag
from autogluon import TabularPrediction as task

import pandas as pd ## read/write data
import re ## regex search for features name
from sklearn import preprocessing ## standardization
import math ## take log10
import numpy as np ## data handling

In [62]:
!pip3 list


Package                 Version             
----------------------- --------------------
absl-py                 0.11.0              
alembic                 1.4.2               
apturl                  0.5.2               
astunparse              1.6.3               
async-generator         1.10                
attrs                   19.3.0              
autogluon               0.0.13              
autogluon-contrib-nlp   0.0.1b20200815      
autokeras               1.0.12              
Automat                 0.8.0               
backcall                0.1.0               
bcrypt                  3.1.7               
bleach                  3.1.5               
blinker                 1.4                 
boto3                   1.14.60             
botocore                1.17.60             
Brlapi                  0.7.0               
cachetools              4.1.1               
catboost                0.23.2              
certifi                 2020.6.20           
certipy   

In [16]:
### if consider padel features ,then remove NO 281 / 295 / 1046 dat
def getFeatures(dat,fp=True,padel=True,fpType = "circular",fp_output = False,removeUselessFp=True):
    if fp:
    # trans fingerprint
        if ~fp_output:
            fps = pd.read_table((dataPath + "fp_" + fpType + ".tsv"), sep = "\t")
        if removeUselessFp:
            varFps = fps.apply(lambda x: ((max(x)-min(x)) != 0),axis=0)
            fps = fps[list(varFps[varFps].index)]
        dat = pd.merge(dat, fps, on ="No", how="outer")
    if padel: #PaDEL dat - remove features which has missing dat (keep 612 features)
        padelDat = pd.read_excel(dataPath + "data.xlsx",sheet_name= "padel-neuro_network")
        padelDat.dropna(axis = 1, how = 'any', inplace = True)
        dat = pd.merge(dat, padelDat, left_on ="No", right_on ="Name", how="outer")
        dat = dat.drop(columns=["Name"])
    return dat

In [11]:
dataPath = "./Project/zebrafish/MOST2021/"
data = pd.read_excel(dataPath + "data.xlsx",sheet_name= "analyzable")
data = data.drop(columns=["DNT-Label","DNT-Label2","CAESAR"])
dat = getFeatures(data,fp=True,padel=True,fpType = "circular",fp_output = False,removeUselessFp=True)

In [36]:
fp = True
padel = True
batch_size = dat.shape[0]
n_train = 3 
n_validate = 1
n_test= 1
MASK_VALUE = '-999'

In [78]:
for fishFeatures in list(dat.columns[6:30]):
    fishFeatures = [fishFeatures]
    print(str(fishFeatures))

['MO24']
['DP24']
['SM24']
['NC24']
['MORT']
['YSE']
['AXIS']
['EYE']
['SNOU']
['JAW']
['OTIC']
['PE']
['BRAI']
['SOMI']
['PFIN']
['CFIN']
['PIG']
['CIRC']
['TRUN']
['SWIM']
['NC']
['TR']
['MOV21']
['AUC21']


In [79]:
for fishFeatures in list(dat.columns[6:30]):
    fishFeatures = [fishFeatures]
    ## select features
    chemicalInfo = list(dat.columns[0:6])
    fpFeatures = list(filter(re.compile("^V[0-9]+$").match, list(dat.columns)))
    ppFeatures = list(set(list(dat.columns)) - set(chemicalInfo) - set(fishFeatures) - set(fpFeatures))
    selectFeatures = list()
    if fp:
        selectFeatures = selectFeatures + fpFeatures
    if padel:
        selectFeatures = selectFeatures + ppFeatures
    scaler = preprocessing.StandardScaler()
    x = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame()] # x[0]:training / x[1]:validate / x[2]:test
    y = [[],[],[]] # y[0]:training{y[0] = [[task0,MASK_VALUE],...,[MASK_VALUE,taskn]]} / y[1]:validate / y[2]:test
    batch_et = math.ceil(batch_size/len(fishFeatures))
    for i, target in enumerate(fishFeatures):
        tmpDat = dat[selectFeatures + [target]].rename(columns={target: 'target'})
        tmpDat[["target"]] = tmpDat[["target"]].applymap(lambda x: math.log10(x)) ## target take log10
        tmpDat = tmpDat.sort_values(["target"], ascending=[0])
        tmpDat["trainTest"] = np.tile( \
          np.repeat(["training","validate","test"], [n_train, n_validate,n_test], axis=0) \
          ,(math.ceil(tmpDat.shape[0]/(n_train+ n_test))))[0:tmpDat.shape[0]]
        if padel: # standardize ppFeatures
            tmpDat_way = [[],[],[]]
            for j, way in enumerate(["training","validate","test"]):
                tmpDat_way[j] = tmpDat[tmpDat["trainTest"]== way ]
                if way == "training":
                    new_pp = pd.DataFrame(scaler.fit_transform(tmpDat_way[0][ppFeatures].values), index=tmpDat_way[0][ppFeatures].index, columns=tmpDat_way[0][ppFeatures].columns)    
                else:
                    new_pp = pd.DataFrame(scaler.transform(tmpDat_way[j][ppFeatures].values), index=tmpDat_way[j][ppFeatures].index, columns=tmpDat_way[j][ppFeatures].columns)
                tmpDat_way[j] = tmpDat_way[j].drop(columns=ppFeatures)
                tmpDat_way[j] = pd.concat([tmpDat_way[j], new_pp], axis=1)
            tmpDat = tmpDat_way[0].append(tmpDat_way[1],ignore_index=True)
            tmpDat = tmpDat.append(tmpDat_way[2],ignore_index=True)
        for j, way in enumerate(["training","validate","test"]):
            x_tmp = tmpDat[tmpDat["trainTest"]== way ][selectFeatures]
            x_tmp["batch_group"] = np.tile(\
                range(math.ceil(tmpDat[tmpDat["trainTest"]== way ].shape[0]/batch_et)),\
                tmpDat[tmpDat["trainTest"]== way ].shape[0])[0:tmpDat[tmpDat["trainTest"]== way ].shape[0]]
            x[j] = x[j].append(x_tmp)
            tmp_target = np.full([tmpDat[tmpDat["trainTest"]== way ]["target"].shape[0],len(fishFeatures)+1],MASK_VALUE)
            tmp_target[:,i] = tmpDat[tmpDat["trainTest"]== way ]["target"].values
            tmp_target[:,len(fishFeatures)] = x_tmp["batch_group"]
            y[j].extend(tmp_target)
    for i in range(3):
        x[i] = np.array(x[i])[np.array(x[i])[:,-1].argsort()]
        x[i] = np.array(x[i])[:,0:(np.array(x[i]).shape[1]-1)]
        y[i] = np.array(y[i])[np.array(y[i])[:,-1].argsort()]
        y[i] = np.array(y[i])[:,0:(np.array(y[i]).shape[1]-1)]
    fname = ["train","val","test"]
    for i in range(3):
        tmp = np.concatenate((x[i], y[i]), axis=1)
        np.savetxt(str(fishFeatures)+(fname[i]+".csv"), tmp,'%s', delimiter=",", header=(",".join(selectFeatures)+",target"))
    train_data = task.Dataset(file_path=str(fishFeatures)+'train.csv')
    val_data = task.Dataset(file_path=str(fishFeatures)+'val.csv')
    test_data = task.Dataset(file_path=str(fishFeatures)+'test.csv')
    label_column = 'target'
    logdir = 'agModels-' + str(fishFeatures)  # specifies folder where to store trained models
    print(str(fishFeatures))
    predictor = task.fit(train_data=train_data, tuning_data=val_data,
                         problem_type = 'regression', eval_metric = 'mean_absolute_error',
                         hyperparameter_tune = True, num_bagging_folds = 5,
                         label=label_column, output_directory=logdir,
                        )

Loaded data from: ['MO24']train.csv | Columns = 1672 / 1672 | Rows = 645 -> 645
Loaded data from: ['MO24']val.csv | Columns = 1672 / 1672 | Rows = 215 -> 215
Loaded data from: ['MO24']test.csv | Columns = 1672 / 1672 | Rows = 214 -> 214
Beginning AutoGluon training ...
AutoGluon will save models to agModels-['MO24']/
AutoGluon Version:  0.0.13
Train Data Rows:    645
Train Data Columns: 1672
Tuning Data Rows:    215
Tuning Data Columns: 1672
Preprocessing data ...


['MO24']


Feature Generator processed 860 data points with 1671 features
Original Features (raw dtypes):
	float64 features: 1671
Original Features (inferred dtypes):
	float features: 1671
Generated Features (special dtypes):
Processed Features (raw dtypes):
	float features: 1671
Processed Features:
	float features: 1671
	Data preprocessing and feature engineering runtime = 2.58s ...
AutoGluon will gauge predictive performance using evaluation metric: mean_absolute_error
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: mean_absolute_error
	-1.3008	 = Validation mean_absolute_error score
	6.64s	 = Training runtime
	0.37s	 = Validation runtime
	-1.2679	 = Validation mean_absolute_error score
	3.52s	 = Training runtime
	0.4s	 = Validation runtime
	-1.4377	 = Validation mean_absolute_error score
	0.09s	 = Training runtime
	0.1s	 = Validation runtime
	-1.4456	 = Validation mean_absolute_error score
	0.12s	 = Training runtime
	0.11s	 = 

Exception ignored in: <function tqdm.__del__ at 0x7febc4de58b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1090, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/tqdm/notebook.py", line 255, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm' object has no attribute 'sp'
Fitting model: RandomForestRegressorMSE_STACKER_l0/0 ...
	-1.2157	 = Validation mean_absolute_error score
	32.91s	 = Training runtime
	2.26s	 = Validation runtime
Fitting model: ExtraTreesRegressorMSE_STACKER_l0/0 ...
	-1.2321	 = Validation mean_absolute_error score
	17.63s	 = Training runtime
	2.24s	 = Validation runtime
Fitting model: KNeighborsRegressorUnif_STACKER_l0/0 ...
	-1.3439	 = Validation mean_absolute_error score
	0.54s	 = Training runtime
	0.5s	 = Validation runtime
Fitting model: KNeighborsRegressorDist_STACKER_l0/0 ...
	-1.3427	 = Validation mean_absolute_error score
	0.55s	 = Training runtime
	0.53s	 = Valida

['DP24']


...

Feature Generator processed 860 data points with 1671 features
Original Features (raw dtypes):
	float64 features: 1671
Original Features (inferred dtypes):
	float features: 1671
Generated Features (special dtypes):
Processed Features (raw dtypes):
	float features: 1671
Processed Features:
	float features: 1671
	Data preprocessing and feature engineering runtime = 2.21s ...
AutoGluon will gauge predictive performance using evaluation metric: mean_absolute_error
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: mean_absolute_error
	-2.6312	 = Validation mean_absolute_error score
	5.89s	 = Training runtime
	0.49s	 = Validation runtime
	-2.5518	 = Validation mean_absolute_error score
	3.49s	 = Training runtime
	0.45s	 = Validation runtime
	-2.7446	 = Validation mean_absolute_error score
	0.09s	 = Training runtime
	0.07s	 = Validation runtime
	-2.7517	 = Validation mean_absolute_error score
	0.08s	 = Training runtime
	0.08s	 

Exception ignored in: <function tqdm.__del__ at 0x7febc4de58b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1090, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/tqdm/notebook.py", line 255, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm' object has no attribute 'sp'
Fitting model: RandomForestRegressorMSE_STACKER_l0/0 ...
	-2.6764	 = Validation mean_absolute_error score
	29.57s	 = Training runtime
	2.28s	 = Validation runtime
Fitting model: ExtraTreesRegressorMSE_STACKER_l0/0 ...
	-2.7281	 = Validation mean_absolute_error score
	17.93s	 = Training runtime
	2.39s	 = Validation runtime
Fitting model: KNeighborsRegressorUnif_STACKER_l0/0 ...
	-2.902	 = Validation mean_absolute_error score
	0.52s	 = Training runtime
	0.49s	 = Validation runtime
Fitting model: KNeighborsRegressorDist_STACKER_l0/0 ...
	-2.9122	 = Validation mean_absolute_error score
	0.43s	 = Training runtime
	0.5s	 = Validat

['AUC21']


Feature Generator processed 860 data points with 1671 features
Original Features (raw dtypes):
	float64 features: 1671
Original Features (inferred dtypes):
	float features: 1671
Generated Features (special dtypes):
Processed Features (raw dtypes):
	float features: 1671
Processed Features:
	float features: 1671
	Data preprocessing and feature engineering runtime = 2.16s ...
AutoGluon will gauge predictive performance using evaluation metric: mean_absolute_error
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: mean_absolute_error
	-2.5357	 = Validation mean_absolute_error score
	6.42s	 = Training runtime
	0.46s	 = Validation runtime
	-2.8972	 = Validation mean_absolute_error score
	3.7s	 = Training runtime
	0.54s	 = Validation runtime
	-2.8229	 = Validation mean_absolute_error score
	0.11s	 = Training runtime
	0.1s	 = Validation runtime
	-2.8442	 = Validation mean_absolute_error score
	0.08s	 = Training runtime
	0.09s	 = 

Exception ignored in: <function tqdm.__del__ at 0x7febc4de58b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1090, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/tqdm/notebook.py", line 255, in close
    self.sp(bar_style='danger')
AttributeError: 'tqdm' object has no attribute 'sp'
Fitting model: RandomForestRegressorMSE_STACKER_l0/0 ...
	-2.5862	 = Validation mean_absolute_error score
	31.8s	 = Training runtime
	2.32s	 = Validation runtime
Fitting model: ExtraTreesRegressorMSE_STACKER_l0/0 ...
	-2.7149	 = Validation mean_absolute_error score
	19.02s	 = Training runtime
	2.53s	 = Validation runtime
Fitting model: KNeighborsRegressorUnif_STACKER_l0/0 ...
	-2.8056	 = Validation mean_absolute_error score
	0.56s	 = Training runtime
	0.5s	 = Validation runtime
Fitting model: KNeighborsRegressorDist_STACKER_l0/0 ...
	-2.8179	 = Validation mean_absolute_error score
	0.53s	 = Training runtime
	0.51s	 = Validat